In [ ]:
#| default_exp actions

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from triggerkit import util
import functools
import inspect
import uuid
from typing import Dict, List, Callable, Optional, Any, Union

# actions

When your views yields results, what **TriggerKit** performs are **actions**. These are functions that take in results and perform things like send slack messages, perform SQL operations, make API calls, etc. 

You can either use some of our built-in actions or build your own!

## Built In Actions:

In [ ]:
#| export

def send_slack_message(data): 
    """
    Use the returned data to send slack messages. This could be used for creating alerts for data issues, reporting on metrics, etc.
    """
    pass

def run_sql(data): 
    pass

## Make Your Own Actions

It is simple to make your own actions. All you have to do is make a function that takes in a List of Dicts and `register` it with the system


Example action:

```python
from triggerkit import register

@register('My Action','Does something I want')
def my_action(
    data: List[Dict],               # List of results from the view
    context: Dict[str, Any],        # Shared context between actions
    config: Dict[str, Any],         # Config for the action - Passes job-specific parameters (e.g. thresholds, channel IDs)
    job_name: str,                  # (Optional) Name of the job  - Helpful for logging, auditing, and debugging
    run_id: str,                    # (Optional) Unqiue ID of the run - Useful for traceability in logs, alerts, and audits
    **kwargs
) -> Dict[str, Any]:
    ...

In [ ]:
#| export
util.ACTION_REGISTRY: Dict[str, Callable] = {}

# Standard action schema
ACTION_SCHEMA = {
    "data": {"type": "List[Dict]", "description": "Rows from the Snowflake view."},
    "context": {"type": "Dict[str, Any]", "description": "Shared context between actions."},
    "job_name": {"type": "str", "description": "Name of the job triggering this action.","optional": True},
    "run_id": {"type": "str", "description": "Unique ID for this run.","optional": True},
}

def _gen_action_doc(func,schema):
    lines = ["Arguments:", ""]
    sig = inspect.signature(func)
    for param in sig.parameters.values():
        if param.name not in schema:
            lines.append(f"- `{param.name}` ({param.annotation.__name__}): {param.default}")
        else:
            meta = schema[param.name]
            opt = " (optional)" if meta.get("optional") else ""
            lines.append(f"- `{param.name}` ({meta['type']}){opt}: {meta['description']}")
    return "\n".join(lines)

def _validate_action_function(func, schema):
    sig = inspect.signature(func)
    missing = [
        k for k in schema
        if k not in sig.parameters and not schema[k].get("optional")
    ]
    if missing:
        raise ValueError(f"Action '{func.__name__}' is missing required argument(s): {missing}")



In [ ]:
#| hide

# Checking the docstring gerenated from the function

def example_action(data, context, config, job_name, run_id=None, dry_run=False, unique_id: int=None):
    pass

def check_gen_doc(fn,name,description=None):
    if not fn.__doc__:
            fn.__doc__ = _gen_action_doc(fn,ACTION_SCHEMA)
    
    return fn

example_action = check_gen_doc(example_action,"example_action","Example action function.")
help(example_action)

Help on function example_action in module __main__:

example_action(data, context, config, job_name, run_id=None, dry_run=False, unique_id: int = None)
    Arguments:
    
    - `data` (List[Dict]): Rows from the Snowflake view.
    - `context` (Dict[str, Any]): Shared context between actions.
    - `config` (_empty): <class 'inspect._empty'>
    - `job_name` (str) (optional): Name of the job triggering this action.
    - `run_id` (str) (optional): Unique ID for this run.
    - `dry_run` (_empty): False
    - `unique_id` (int): None



In [ ]:
#| export

def register(name: str, description: Optional[str] = None, overwrite: bool = False) -> Callable:
    """
    Register your function as an action in the global registry.
    
    **Args**: 
        \n • `name`: Descriptive name for your action
        \n • `description`: Optional description of what the action does

    **Raises**:
       \n • `ValueError`: If an action with the same name already exists
    """
    def decorator(fn: Callable) -> Callable:
        _validate_action_function(fn, ACTION_SCHEMA)

        if name in util.ACTION_REGISTRY and util.ACTION_REGISTRY[name] != fn and not overwrite:
            util.logger.warning(f"Action '{name}' already exists. Keeping the existing one.")
            return fn
        if name in util.ACTION_REGISTRY and overwrite:
            util.logger.warning(f"Overwriting existing action '{name}' with new one.")
            # Remove the old action
            del util.ACTION_REGISTRY[name]
        
        if not fn.__doc__: fn.__doc__ =  _gen_action_doc(fn,ACTION_SCHEMA)
        
        # Add metadata to the function
        fn._action_name = name
        fn._action_description = description
        
        # Add to registry
        util.ACTION_REGISTRY[name] = fn
        
        # Preserve function metadata like docstrings
        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            return fn(*args, **kwargs)
        
        return wrapper
    util.logger.info(f"Registered action: {name}")
    return decorator

In [ ]:
#| export

def list_available():
    """List all registered actions with their descriptions."""
    print("Available actions:")
    for name, fn in util.ACTION_REGISTRY.items():
        description = getattr(fn, "_action_description", "No description available")
        print(f"  - {name}: {description}")
    return util.ACTION_REGISTRY

def get_info(name: str) -> Dict[str, Any]:
    """Get detailed information about a specific action."""
    if name not in util.ACTION_REGISTRY:
        raise ValueError(f"No action named '{name}' is registered.")
    
    fn = util.ACTION_REGISTRY[name]
    info = {
        "name": name,
        "description": getattr(fn, "_action_description", "No description available"),
        "docstring": fn.__doc__ or "No documentation",
        "signature": str(inspect.signature(fn))
    }
    return info

In [ ]:
#| export

def run(action_name: str, 
        data: List[Dict[str, Any]],
        context: Optional[Dict[str, Any]] = None,
        config: Optional[Dict[str, Any]] = None,
        job_name: Optional[str] = None,
        dry_run: Optional[bool] = False
        ) -> Any:
    """
    Run a registered action with the provided data.
    
    **Args:** 
        \n • `action_name`: Name of the registered action
        \n • `data`: Data to pass to the action
        
    **Returns:**
        Result of the action
    """
    if action_name not in util.ACTION_REGISTRY:
        raise ValueError(f"No action named '{action_name}' is registered")
    
    action = util.ACTION_REGISTRY[action_name]

    try:
        params = {  "data": data,
                    "context": context or {},
                    "config": config or {},
                    "job_name": job_name,
                    "run_id": str(uuid.uuid4()),
                    "dry_run": dry_run}
        
        # Call the action with the parameters it accepts
        sig = inspect.signature(action)
        accepted_params = {k: v for k, v in params.items() if k in sig.parameters}

        result = action(**accepted_params)
    except Exception as e:
        util.logger.error(f"Failed to execute action '{action_name}': {str(e)}")
    else:
        util.logger.info(f"Action '{action_name}' completed successfully")
        return result

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()